# Character-Level LSTM in PyTorch

In this notebook, I'll construct a character-level LSTM with PyTorch. The network will train character by character on some text, then generate new text character by character. As an example, I will train on Anna Karenina. **This model will be able to generate new text based on the text from the book!**

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

First let's load in our required resources for data loading and model creation.

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Load in Data

Then, we'll load the Anna Karenina text file and convert it into integers for our network to use. 

In [2]:
# open text file and read in data as `text`
with open('data/anna.txt', 'r') as f:
    text = f.read()

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

### Tokenization

In the cells, below, I'm creating a couple of **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [4]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

And we can see those same characters from above, encoded as integers.

In [5]:
encoded[:100]

array([49, 37, 17, 25, 75,  4, 63, 41, 73, 23, 23, 23, 59, 17, 25, 25, 44,
       41, 31, 17, 32, 68, 33, 68,  4, 66, 41, 17, 63,  4, 41, 17, 33, 33,
       41, 17, 33, 68, 24,  4, 81, 41,  4, 57,  4, 63, 44, 41, 52, 60, 37,
       17, 25, 25, 44, 41, 31, 17, 32, 68, 33, 44, 41, 68, 66, 41, 52, 60,
       37, 17, 25, 25, 44, 41, 68, 60, 41, 68, 75, 66, 41, 22, 29, 60, 23,
       29, 17, 44, 30, 23, 23,  3, 57,  4, 63, 44, 75, 37, 68, 60])

## Pre-processing the data

As you can see in our char-RNN image above, our LSTM expects an input that is **one-hot encoded** meaning that each character is converted into an integer (via our created dictionary) and *then* converted into a column vector where only it's corresponding integer index will have the value of 1 and the rest of the vector will be filled with 0's. Since we're one-hot encoding the data, let's make a function to do that!


In [6]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [7]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [8]:
print(one_hot.shape)

(1, 3, 8)


## Making training mini-batches


To train on this data, we also want to create mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

In this example, we'll take the encoded characters (passed in as the `arr` parameter) and split them into multiple sequences, given by `batch_size`. Each of our sequences will be `seq_length` long.

### Creating Batches

**1. The first thing we need to do is discard some of the text so we only have completely full mini-batches. **

Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences in a batch) and $M$ is the seq_length or number of time steps in a sequence. Then, to get the total number of batches, $K$, that we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

**2. After that, we need to split `arr` into $N$ batches. ** 

You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences in a batch, so let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

**3. Now that we have this array, we can iterate through it to get our mini-batches. **

The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `seq_length`. We also want to create both the input and target arrays. Remember that the targets are just the inputs shifted over by one character. The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of tokens in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `seq_length` wide.

> **TODO:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [9]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    ## TODO: Get the number of batches we can make
    n_batches = len(arr) // batch_size
    
    ## TODO: Keep only enough characters to make full batches
    arr = arr[:n_batches*batch_size]
    
    ## TODO: Reshape into batch_size rows
    arr = arr.reshape(batch_size, -1)
    
    ## TODO: Iterate over the batches using a window of size seq_length
    # Here we are creating non-overlapping sequences
    # 1,2,3,4,5,6
    # n-> 0, , seq_len = 3
    # In my apporach, I am breaking continuity of the sequence whereas in the solution
    # It preserves the continuity
    for n in range(0, arr.shape[1]-seq_length-1, seq_length):
        # The features
        x = arr[:,n:n+seq_length]
        # The targets, shifted by one
        y = arr[:,n+1:n+seq_length+1]
        yield x, y

### Test Your Implementation

Now I'll make some data sets and we can check out what's going on as we batch data. Here, as an example, I'm going to use a batch size of 8 and 50 sequence steps.

In [10]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [11]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[49 37 17 25 75  4 63 41 73 23]
 [60 41 75 37 17 75 41 17 75 75]
 [22 63 41 17 41 31 22  4 42 41]
 [48 37 68  4 31 41 75 37 22 52]
 [41 75  4 17 63 55 66 75 17 68]
 [53 41 17 60 17 33 44 66 68 66]
 [68 53 41 75 37 17 75 41 26 22]
 [41 71 75 37  4 44 71 41 48 17]]

y
 [[37 17 25 75  4 63 41 73 23 23]
 [41 75 37 17 75 41 17 75 75 63]
 [63 41 17 41 31 22  4 42 41 37]
 [37 68  4 31 41 75 37 22 52 16]
 [75  4 17 63 55 66 75 17 68 60]
 [41 17 60 17 33 44 66 68 66 42]
 [53 41 75 37 17 75 41 26 22 33]
 [71 75 37  4 44 71 41 48 17 60]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[25  8 60 11 45 27 28 73  1  2]
 [17  7 20 73 45  8 60 45 73 60]
 [27 20 80 73  7 28 73 60 73 65]
 [17 73 45  8 27 73 66  8 46 27]
 [73 17 60 12 73  8 27 28 73 45]
 [66 64 17 17 46  7 20 73 60 20]
 [73 76 20 20 60 73  8 60 80 73]
 [47 35 43  7 20 17 24 50 37 73]]

y
 [[ 8 60 11 45 27 28 73  1  2  2]
 [ 7 20 73 45  8 60 45 73 60 45]
 [20 80 73  7 28 73 60 73 65  7]
 [73 45  8 27 73 66  8 46 27 65]
 [17 60 12 73  8 27 28 73 45 27]
 [64 17 17 46  7 20 73 60 20 80]
 [76 20 20 60 73  8 60 80 73 17]
 [35 43  7 20 17 24 50 37 73 36]]
 ```
 although the exact numbers may be different. Check to make sure the data is shifted over one step for `y`.

---
## Defining the network with PyTorch

Below is where you'll define the network.

<img src="assets/charRNN.png" width=500px>

Next, you'll use PyTorch to define the architecture of the network. We start by defining the layers and operations we want. Then, define a method for the forward pass. You've also been given a method for predicting characters.

### Model Structure

In `__init__` the suggested structure is as follows:
* Create and store the necessary dictionaries (this has been done for you)
* Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size `n_hidden`, a number of layers `n_layers`, a dropout probability `drop_prob`, and a batch_first boolean (True, since we are batching)
* Define a dropout layer with `drop_prob`
* Define a fully-connected layer with params: input size `n_hidden` and output size (the number of characters)
* Finally, initialize the weights (again, this has been given)

Note that some parameters have been named and given in the `__init__` function, and we use them and store them by doing something like `self.drop_prob = drop_prob`.

---
### LSTM Inputs/Outputs

You can create a basic [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) as follows

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

where `input_size` is the number of characters this cell expects to see as sequential input, and `n_hidden` is the number of units in the hidden layers in the cell. And we can add dropout by adding a dropout parameter with a specified probability; this will automatically add dropout to the inputs or outputs. Finally, in the `forward` function, we can stack up the LSTM cells into layers using `.view`. With this, you pass in a list of cells and it will send the output of one cell into the next cell.

We also need to create an initial hidden state of all zeros. This is done like so

```python
self.init_hidden()
```

In [12]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [13]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.output_len = len(self.chars)
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the layers of the model
        self.lstm = nn.LSTM(input_size=self.output_len, hidden_size = n_hidden, num_layers=n_layers,\
                            dropout=drop_prob, batch_first=True)
        self.output_layer = nn.Linear(self.n_hidden, self.output_len)
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        lstm_out, hidden = self.lstm(x, hidden)
        lstm_out = lstm_out.reshape(-1,self.n_hidden)
        # return the final output and the hidden state
        out = self.output_layer(lstm_out)
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # Two vectors- one for long term memory and the other one for short term memory
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

## Time to train

The train function gives us the ability to set the number of epochs, the learning rate, and other parameters.

Below we're using an Adam optimizer and cross entropy loss since we are looking at character class scores as output. We calculate the loss and perform backpropagation, as usual!

A couple of details about training: 
>* Within the batch loop, we detach the hidden state from its history; this time setting it equal to a new *tuple* variable because an LSTM has a hidden state that is a tuple of the hidden and cell states.
* We use [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) to help prevent exploding gradients.

In [14]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            # optim.zero_grad() vs net.zero_grad()
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            ## output- [B*T,V], target-[B,T,V]
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()
                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Instantiating the model

Now we can actually train the network. First we'll create the network itself, with some given hyperparameters. Then, define the mini-batches sizes, and start training!

In [19]:
## TODO: set your model hyperparameters
# define and print the net
n_hidden=128
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 128, num_layers=2, batch_first=True, dropout=0.5)
  (output_layer): Linear(in_features=128, out_features=83, bias=True)
)


### Set your training hyperparameters!

In [20]:
batch_size = 64
# Average word length is 4, It is good if we want to learn the next character given a singel word
# but the next character may depend upon the whole sentence
# The average sentence length is 15-20. Which is good if we want to learn next character given the sentence
# Let's take a middle gorund 10
seq_length = 10
n_epochs =  20# start small if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/20... Step: 10... Loss: 4.1746... Val Loss: 4.0260
Epoch: 1/20... Step: 20... Loss: 3.1985... Val Loss: 3.1771
Epoch: 1/20... Step: 30... Loss: 3.1682... Val Loss: 3.1466
Epoch: 1/20... Step: 40... Loss: 3.0634... Val Loss: 3.1352
Epoch: 1/20... Step: 50... Loss: 3.0565... Val Loss: 3.1325
Epoch: 1/20... Step: 60... Loss: 3.1415... Val Loss: 3.1323
Epoch: 1/20... Step: 70... Loss: 3.1558... Val Loss: 3.1316
Epoch: 1/20... Step: 80... Loss: 3.0819... Val Loss: 3.1312
Epoch: 1/20... Step: 90... Loss: 3.0850... Val Loss: 3.1291
Epoch: 1/20... Step: 100... Loss: 3.1406... Val Loss: 3.1305
Epoch: 1/20... Step: 110... Loss: 3.0404... Val Loss: 3.1279
Epoch: 1/20... Step: 120... Loss: 3.0680... Val Loss: 3.1297
Epoch: 1/20... Step: 130... Loss: 3.1020... Val Loss: 3.1296
Epoch: 1/20... Step: 140... Loss: 3.1149... Val Loss: 3.1288
Epoch: 1/20... Step: 150... Loss: 3.0521... Val Loss: 3.1336
Epoch: 1/20... Step: 160... Loss: 3.1481... Val Loss: 3.1312
Epoch: 1/20... Step: 170... Loss:

Epoch: 1/20... Step: 1350... Loss: 2.1975... Val Loss: 2.2360
Epoch: 1/20... Step: 1360... Loss: 2.2813... Val Loss: 2.2394
Epoch: 1/20... Step: 1370... Loss: 2.2583... Val Loss: 2.2255
Epoch: 1/20... Step: 1380... Loss: 2.1590... Val Loss: 2.2176
Epoch: 1/20... Step: 1390... Loss: 2.1874... Val Loss: 2.2197
Epoch: 1/20... Step: 1400... Loss: 2.2630... Val Loss: 2.2166
Epoch: 1/20... Step: 1410... Loss: 2.1361... Val Loss: 2.2085
Epoch: 1/20... Step: 1420... Loss: 2.1374... Val Loss: 2.2057
Epoch: 1/20... Step: 1430... Loss: 2.1584... Val Loss: 2.2053
Epoch: 1/20... Step: 1440... Loss: 2.2272... Val Loss: 2.2082
Epoch: 1/20... Step: 1450... Loss: 2.1139... Val Loss: 2.2015
Epoch: 1/20... Step: 1460... Loss: 2.1514... Val Loss: 2.1972
Epoch: 1/20... Step: 1470... Loss: 2.1635... Val Loss: 2.1916
Epoch: 1/20... Step: 1480... Loss: 2.1057... Val Loss: 2.1942
Epoch: 1/20... Step: 1490... Loss: 2.1343... Val Loss: 2.1898
Epoch: 1/20... Step: 1500... Loss: 2.1980... Val Loss: 2.1862
Epoch: 1

Epoch: 1/20... Step: 2680... Loss: 1.8392... Val Loss: 1.9488
Epoch: 1/20... Step: 2690... Loss: 1.8610... Val Loss: 1.9452
Epoch: 1/20... Step: 2700... Loss: 1.9363... Val Loss: 1.9478
Epoch: 1/20... Step: 2710... Loss: 1.8423... Val Loss: 1.9455
Epoch: 1/20... Step: 2720... Loss: 1.9405... Val Loss: 1.9469
Epoch: 1/20... Step: 2730... Loss: 1.9751... Val Loss: 1.9411
Epoch: 1/20... Step: 2740... Loss: 1.8825... Val Loss: 1.9460
Epoch: 1/20... Step: 2750... Loss: 1.8500... Val Loss: 1.9382
Epoch: 1/20... Step: 2760... Loss: 1.8501... Val Loss: 1.9384
Epoch: 1/20... Step: 2770... Loss: 1.8629... Val Loss: 1.9364
Epoch: 1/20... Step: 2780... Loss: 1.8410... Val Loss: 1.9372
Epoch: 1/20... Step: 2790... Loss: 1.8819... Val Loss: 1.9336
Epoch: 2/20... Step: 2800... Loss: 1.9112... Val Loss: 1.9354
Epoch: 2/20... Step: 2810... Loss: 1.8463... Val Loss: 1.9279
Epoch: 2/20... Step: 2820... Loss: 1.8538... Val Loss: 1.9288
Epoch: 2/20... Step: 2830... Loss: 1.8163... Val Loss: 1.9276
Epoch: 2

Epoch: 2/20... Step: 4010... Loss: 1.7421... Val Loss: 1.7951
Epoch: 2/20... Step: 4020... Loss: 1.7437... Val Loss: 1.7945
Epoch: 2/20... Step: 4030... Loss: 1.7625... Val Loss: 1.7969
Epoch: 2/20... Step: 4040... Loss: 1.8413... Val Loss: 1.7960
Epoch: 2/20... Step: 4050... Loss: 1.8277... Val Loss: 1.7978
Epoch: 2/20... Step: 4060... Loss: 1.7757... Val Loss: 1.7982
Epoch: 2/20... Step: 4070... Loss: 1.7097... Val Loss: 1.7909
Epoch: 2/20... Step: 4080... Loss: 1.6910... Val Loss: 1.7898
Epoch: 2/20... Step: 4090... Loss: 1.6727... Val Loss: 1.7982
Epoch: 2/20... Step: 4100... Loss: 1.6512... Val Loss: 1.7916
Epoch: 2/20... Step: 4110... Loss: 1.7748... Val Loss: 1.7925
Epoch: 2/20... Step: 4120... Loss: 1.7668... Val Loss: 1.7887
Epoch: 2/20... Step: 4130... Loss: 1.5774... Val Loss: 1.7885
Epoch: 2/20... Step: 4140... Loss: 1.7715... Val Loss: 1.7891
Epoch: 2/20... Step: 4150... Loss: 1.8255... Val Loss: 1.7888
Epoch: 2/20... Step: 4160... Loss: 1.8140... Val Loss: 1.7854
Epoch: 2

Epoch: 2/20... Step: 5340... Loss: 1.6160... Val Loss: 1.7195
Epoch: 2/20... Step: 5350... Loss: 1.7294... Val Loss: 1.7178
Epoch: 2/20... Step: 5360... Loss: 1.5604... Val Loss: 1.7169
Epoch: 2/20... Step: 5370... Loss: 1.5969... Val Loss: 1.7157
Epoch: 2/20... Step: 5380... Loss: 1.6557... Val Loss: 1.7121
Epoch: 2/20... Step: 5390... Loss: 1.6130... Val Loss: 1.7137
Epoch: 2/20... Step: 5400... Loss: 1.6353... Val Loss: 1.7172
Epoch: 2/20... Step: 5410... Loss: 1.6657... Val Loss: 1.7126
Epoch: 2/20... Step: 5420... Loss: 1.6501... Val Loss: 1.7148
Epoch: 2/20... Step: 5430... Loss: 1.7045... Val Loss: 1.7125
Epoch: 2/20... Step: 5440... Loss: 1.5514... Val Loss: 1.7116
Epoch: 2/20... Step: 5450... Loss: 1.6488... Val Loss: 1.7069
Epoch: 2/20... Step: 5460... Loss: 1.5214... Val Loss: 1.7099
Epoch: 2/20... Step: 5470... Loss: 1.5811... Val Loss: 1.7105
Epoch: 2/20... Step: 5480... Loss: 1.5847... Val Loss: 1.7081
Epoch: 2/20... Step: 5490... Loss: 1.6884... Val Loss: 1.7077
Epoch: 2

...

Epoch: 18/20... Step: 49570... Loss: 1.3580... Val Loss: 1.4435
Epoch: 18/20... Step: 49580... Loss: 1.2785... Val Loss: 1.4477
Epoch: 18/20... Step: 49590... Loss: 1.2803... Val Loss: 1.4480
Epoch: 18/20... Step: 49600... Loss: 1.3338... Val Loss: 1.4470
Epoch: 18/20... Step: 49610... Loss: 1.3308... Val Loss: 1.4460
Epoch: 18/20... Step: 49620... Loss: 1.3837... Val Loss: 1.4462
Epoch: 18/20... Step: 49630... Loss: 1.2553... Val Loss: 1.4446
Epoch: 18/20... Step: 49640... Loss: 1.4431... Val Loss: 1.4438
Epoch: 18/20... Step: 49650... Loss: 1.3377... Val Loss: 1.4408
Epoch: 18/20... Step: 49660... Loss: 1.2893... Val Loss: 1.4408
Epoch: 18/20... Step: 49670... Loss: 1.2621... Val Loss: 1.4422
Epoch: 18/20... Step: 49680... Loss: 1.3215... Val Loss: 1.4426
Epoch: 18/20... Step: 49690... Loss: 1.2957... Val Loss: 1.4423
Epoch: 18/20... Step: 49700... Loss: 1.3098... Val Loss: 1.4445
Epoch: 18/20... Step: 49710... Loss: 1.3348... Val Loss: 1.4436
Epoch: 18/20... Step: 49720... Loss: 1.2

Epoch: 19/20... Step: 50860... Loss: 1.5142... Val Loss: 1.4429
Epoch: 19/20... Step: 50870... Loss: 1.2043... Val Loss: 1.4444
Epoch: 19/20... Step: 50880... Loss: 1.3808... Val Loss: 1.4468
Epoch: 19/20... Step: 50890... Loss: 1.2855... Val Loss: 1.4473
Epoch: 19/20... Step: 50900... Loss: 1.2997... Val Loss: 1.4473
Epoch: 19/20... Step: 50910... Loss: 1.3521... Val Loss: 1.4446
Epoch: 19/20... Step: 50920... Loss: 1.2660... Val Loss: 1.4447
Epoch: 19/20... Step: 50930... Loss: 1.3250... Val Loss: 1.4454
Epoch: 19/20... Step: 50940... Loss: 1.3071... Val Loss: 1.4447
Epoch: 19/20... Step: 50950... Loss: 1.2414... Val Loss: 1.4514
Epoch: 19/20... Step: 50960... Loss: 1.2826... Val Loss: 1.4446
Epoch: 19/20... Step: 50970... Loss: 1.3247... Val Loss: 1.4429
Epoch: 19/20... Step: 50980... Loss: 1.3512... Val Loss: 1.4411
Epoch: 19/20... Step: 50990... Loss: 1.1592... Val Loss: 1.4409
Epoch: 19/20... Step: 51000... Loss: 1.1837... Val Loss: 1.4441
Epoch: 19/20... Step: 51010... Loss: 1.1

Epoch: 19/20... Step: 52150... Loss: 1.2496... Val Loss: 1.4416
Epoch: 19/20... Step: 52160... Loss: 1.3745... Val Loss: 1.4426
Epoch: 19/20... Step: 52170... Loss: 1.3062... Val Loss: 1.4428
Epoch: 19/20... Step: 52180... Loss: 1.2535... Val Loss: 1.4411
Epoch: 19/20... Step: 52190... Loss: 1.2585... Val Loss: 1.4453
Epoch: 19/20... Step: 52200... Loss: 1.2094... Val Loss: 1.4440
Epoch: 19/20... Step: 52210... Loss: 1.3010... Val Loss: 1.4442
Epoch: 19/20... Step: 52220... Loss: 1.4016... Val Loss: 1.4432
Epoch: 19/20... Step: 52230... Loss: 1.3020... Val Loss: 1.4435
Epoch: 19/20... Step: 52240... Loss: 1.3571... Val Loss: 1.4413
Epoch: 19/20... Step: 52250... Loss: 1.3562... Val Loss: 1.4425
Epoch: 19/20... Step: 52260... Loss: 1.2706... Val Loss: 1.4429
Epoch: 19/20... Step: 52270... Loss: 1.2604... Val Loss: 1.4420
Epoch: 19/20... Step: 52280... Loss: 1.3445... Val Loss: 1.4441
Epoch: 19/20... Step: 52290... Loss: 1.2792... Val Loss: 1.4472
Epoch: 19/20... Step: 52300... Loss: 1.3

Epoch: 20/20... Step: 53440... Loss: 1.3211... Val Loss: 1.4416
Epoch: 20/20... Step: 53450... Loss: 1.3589... Val Loss: 1.4404
Epoch: 20/20... Step: 53460... Loss: 1.2541... Val Loss: 1.4402
Epoch: 20/20... Step: 53470... Loss: 1.2300... Val Loss: 1.4413
Epoch: 20/20... Step: 53480... Loss: 1.3427... Val Loss: 1.4419
Epoch: 20/20... Step: 53490... Loss: 1.3345... Val Loss: 1.4388
Epoch: 20/20... Step: 53500... Loss: 1.2934... Val Loss: 1.4390
Epoch: 20/20... Step: 53510... Loss: 1.3365... Val Loss: 1.4403
Epoch: 20/20... Step: 53520... Loss: 1.2056... Val Loss: 1.4397
Epoch: 20/20... Step: 53530... Loss: 1.2725... Val Loss: 1.4404
Epoch: 20/20... Step: 53540... Loss: 1.3420... Val Loss: 1.4409
Epoch: 20/20... Step: 53550... Loss: 1.2934... Val Loss: 1.4420
Epoch: 20/20... Step: 53560... Loss: 1.3220... Val Loss: 1.4394
Epoch: 20/20... Step: 53570... Loss: 1.2907... Val Loss: 1.4407
Epoch: 20/20... Step: 53580... Loss: 1.2446... Val Loss: 1.4401
Epoch: 20/20... Step: 53590... Loss: 1.1

Epoch: 20/20... Step: 54730... Loss: 1.2782... Val Loss: 1.4393
Epoch: 20/20... Step: 54740... Loss: 1.3921... Val Loss: 1.4384
Epoch: 20/20... Step: 54750... Loss: 1.3135... Val Loss: 1.4375
Epoch: 20/20... Step: 54760... Loss: 1.3743... Val Loss: 1.4359
Epoch: 20/20... Step: 54770... Loss: 1.2413... Val Loss: 1.4377
Epoch: 20/20... Step: 54780... Loss: 1.2935... Val Loss: 1.4363
Epoch: 20/20... Step: 54790... Loss: 1.3751... Val Loss: 1.4355
Epoch: 20/20... Step: 54800... Loss: 1.2638... Val Loss: 1.4402
Epoch: 20/20... Step: 54810... Loss: 1.2521... Val Loss: 1.4407
Epoch: 20/20... Step: 54820... Loss: 1.4415... Val Loss: 1.4397
Epoch: 20/20... Step: 54830... Loss: 1.2714... Val Loss: 1.4382
Epoch: 20/20... Step: 54840... Loss: 1.2599... Val Loss: 1.4374
Epoch: 20/20... Step: 54850... Loss: 1.2915... Val Loss: 1.4388
Epoch: 20/20... Step: 54860... Loss: 1.3086... Val Loss: 1.4383
Epoch: 20/20... Step: 54870... Loss: 1.2166... Val Loss: 1.4382
Epoch: 20/20... Step: 54880... Loss: 1.2

## Getting the best model

To set your hyperparameters to get the best performance, you'll want to watch the training and validation losses. If your training loss is much lower than the validation loss, you're overfitting. Increase regularization (more dropout) or use a smaller network. If the training and validation losses are close, you're underfitting so you can increase the size of the network.

## Hyperparameters

Here are the hyperparameters for the network.

In defining the model:
* `n_hidden` - The number of units in the hidden layers.
* `n_layers` - Number of hidden LSTM layers to use.

We assume that dropout probability and learning rate will be kept at the default, in this example.

And in training:
* `batch_size` - Number of sequences running through the network in one pass.
* `seq_length` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lr` - Learning rate for training

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `n_hidden` and `n_layers`. I would advise that you always use `n_layers` of either 2/3. The `n_hidden` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `n_hidden` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

## Checkpoint

After training, we'll save the model so we can load it again later if we need too. Here I'm saving the parameters needed to create the same architecture, the hidden layer hyperparameters and the text characters.

In [21]:
# change the name, for saving multiple files
model_name = '../../saved_models/rnn_2_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Making Predictions

Now that the model is trained, we'll want to sample from it and make predictions about next characters! To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

> To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

### Top K sampling

Our predictions come from a categorical probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text. Read more about [topk, here](https://pytorch.org/docs/stable/torch.html#torch.topk).


In [22]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming and generating text 

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [24]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [28]:
print(sample(net, 1000, prime='Anna said', top_k=5))

Anna said.

"A fancies who have you are completely."

"You meant to tenne, but I wanted, but I'm not thine on the children," he said,
wing to allow and some princess. "Yes, to go to that is think and so much it with his most thing."

"I have a month. The measures. That's not to think," turning this, and the propress to say the feeling on with spite
of
the country and talking to her for
their proffore in his
husbands attention in his watch
with a child. Hooding
her, and answered to
the last, and her
father's family, who had been
a principres of a smile that his face all the carping out with a pationable,
and stood
steps. "I am all my face?..." He said she howered at them. But his hind, and she walked the porter. The chief concession trees to see it, and he had so much. But strok that he was seeing her as
though she so lighted as a strange and threathers after a peasant, so that the conversations.
"We've care a picture," he said.

"A shoulder what you know it with any clans were say at t

## Loading a checkpoint

In [26]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('../../saved_models/rnn_2_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [29]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said is happiness, that they wonked him. He cut his strange of the same wife and that, and, the fack with a little conversation.

"I can't leave you the sort."

"Well, and I didn't call it with all the men more and disapproved," said Stepan
Arkadyevitch was the mind over the same wife and humor to be supporting the friends in the carroa with the chord to the
matchions would have been to listen, when the feeling of the condition
of the farmer
on
the sacrs of the father.

"What as though though he had say no one thank yourself in them. And though I won't tell that," said the same things, before a man of a smiling, that he had to be
already standing her.

This
party as though it wanted to her and, and saw her heart and heart out all the strend, who, and she wanted at the porter and heart, began and cries with them or waiting trackly watching the same doing that she was clirking, and said would have all
strunging.

"I have succeeded this so much it, I cannot be a prince with a li

# Practice Arena

In [4]:
a= "amavs\ns s d"
print(set(a))

{'m', 's', 'v', '\n', 'd', 'a', ' '}


In [14]:
5//3

1